## Customer Support Ticket Classification Test 2 (Ok)
### Pros
* Some Structured data via pydantic models
* Confident Score
### Cons
* We don't give the AI context, Why are there ticket? Who are these tickets for etc. 
* LLM Model have a harder time to answer in a good way without context.

In [ ]:
from pydantic_model import *
from tickets import *
from ollama_connect import *
from IPython.display import display
import pandas as pd
from openai import OpenAI
import instructor

client_simple = OpenAI(
    base_url='http://ollama:11434/v1/',
    api_key='ollama'
)

client_instructor = instructor.patch(
    client_simple,
    mode=instructor.Mode.JSON
)

MODELNAME = "llama3.1:8b"


In [ ]:
def classify_ticket(ticket_text: str) -> TicketClassification:
    response = client_instructor.chat.completions.create(
        model=MODELNAME,
        response_model=TicketClassification,
        temperature=0,
        max_tokens=1024,
        seed=633242,
        top_p=0.2,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": """Analyze the following customer support ticket and extract the requested information.
                            For the 'confidence' field, return a float between 0.0 and 1.0 representing your overall confidence in the accuracy of the category, urgency, and sentiment fields you provide. Never return None."""
            },
            {"role": "user", "content": ticket_text}
        ]
    )
    return response

In [ ]:
rows = []
for i, ticket in enumerate(tickets, 1):
    result = classify_ticket(ticket)

    # Prepare data for DataFrame with cleaned enum values
    data = {
        'category': result.category.value,
        'urgency': result.urgency.value,
        'sentiment': result.sentiment.value,
        'confidence': round(result.confidence, 1) if result.confidence is not None else 0.0,
        'contain_PII_info': round(result.contain_PII_info, 1) if result.contain_PII_info is not None else 0.0,
        'pii_type': result.pii_type.value,
        'key_information': result.key_information,
        'suggested_action': result.suggested_action,
        'raw_ticket': ticket.strip()
    }
    
    rows.append(data)

df = pd.DataFrame(rows)
display(df.style.set_properties(**{'text-align': 'left'}))